In [1]:
import requests
import re
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## CLEANING IMDb_Movies 

#### Read in IMDb_Movies Table

In [2]:
movie_df = pd.read_csv("../Data/Old_Dirty_Data/IMDb_Movies.csv")

In [3]:
movie_df.head(10)

,Unnamed: 0,Title_ID,Title,Director,Rating,Genre,Runtime,Release_Year,Award_Noms,Award_Wins,Score_ID
0,0,17761,The Shawshank Redemption,Frank Darabont,R,Drama,142,1994,43,21,7041
1,1,17762,The Godfather,Francis Ford Coppola,R,"Crime, Drama",175,1972,30,32,7042
2,2,17763,The Dark Knight,Christopher Nolan,PG-13,"Action, Crime, Drama",152,2008,163,160,7043
3,3,17764,The Godfather Part II,Francis Ford Coppola,R,"Crime, Drama",202,1974,21,17,7044
4,4,17765,12 Angry Men,Sidney Lumet,Approved,"Crime, Drama",96,1957,13,17,7045
5,5,17766,Schindler's List,Steven Spielberg,R,"Biography, Drama, History",195,1993,49,91,7046
6,6,17767,The Lord of the Rings: The Return of the King,Peter Jackson,PG-13,"Action, Adventure, Drama",201,2003,124,209,7047
7,7,17768,Pulp Fiction,Quentin Tarantino,R,"Crime, Drama",154,1994,75,70,7048
8,8,17769,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,PG-13,"Action, Adventure, Drama",178,2001,126,121,7049
9,9,177610,"The Good, the Bad and the Ugly",Sergio Leone,R,"Adventure, Western",178,1966,6,3,70410


##### Drop 'Unnamed: 0' which I believe is the index in the csv being imported in

In [4]:
movie_df.pop("Unnamed: 0")

0        0
1        1
2        2
3        3
4        4
      ... 
245    245
246    246
247    247
248    248
249    249
Name: Unnamed: 0, Length: 250, dtype: int64

In [5]:
movie_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title_ID      250 non-null    int64 
 1   Title         250 non-null    object
 2   Director      250 non-null    object
 3   Rating        250 non-null    object
 4   Genre         250 non-null    object
 5   Runtime       250 non-null    int64 
 6   Release_Year  250 non-null    int64 
 7   Award_Noms    250 non-null    int64 
 8   Award_Wins    250 non-null    int64 
 9   Score_ID      250 non-null    int64 
dtypes: int64(6), object(4)
memory usage: 19.7+ KB


#### Next, I need to clean up Ratings and the discrepencies there

Source: Motion Picture Association (MPA)
Link: https://www.motionpictures.org/who-we-are/#our-history
Summary: Founded in 1922, MPA started with the Hays Code system which which simply approved or disapproved based on whether the movies were deemed "moral" or "immoral". In 1968, the Hayes Code was replaced with the modern rating system we have today.
    
Source: IMDb
Link: https://help.imdb.com/article/contribution/titles/certificates/GU757M8ZJ9ZPXB39#
Summary: This website goes through each country and explains the ratings 

From the research that I have conducted, I have decided that this will be the standard to limit the discrepencies in the rating categories.<br>
<bold> DECISION: <bold> Passed and Approved will be combined to be "Approved". Furthermore, Unrated and Not Rated show very little difference in the research and typically does not follow any standard for why one is used over another; thus, these will be combined as "Not Rated"

In [6]:
movie_df['Rating'].value_counts()

R                        101
PG                        37
PG-13                     34
Not Rated                 24
G                         19
Passed                    16
Approved                  13
Not Available on IMDb      1
TV-PG                      1
Unrated                    1
X                          1
TV-MA                      1
GP                         1
Name: Rating, dtype: int64

In [7]:
movie_df['Rating'] = movie_df['Rating'].replace({'GP':'PG', 'Unrated': 'Not Rated', 'Passed':'Approved'})

In [8]:
movie_df['Rating'].value_counts()

R                        101
PG                        38
PG-13                     34
Approved                  29
Not Rated                 25
G                         19
Not Available on IMDb      1
TV-PG                      1
X                          1
TV-MA                      1
Name: Rating, dtype: int64

#### Genre has some misplaced commas between "-" 

In [9]:
movie_df['Genre'] = [x.replace("-, ","-") for x in movie_df['Genre']]

In [10]:
movie_df['Genre'].value_counts()

Drama                           19
Crime, Drama                    14
Animation, Adventure, Comedy    10
Biography, Drama, History        9
Crime, Drama, Mystery            9
                                ..
Action, Thriller                 1
Drama, Mystery, Sci-Fi           1
Comedy, Drama, Family            1
Comedy, Music, Romance           1
Comedy                           1
Name: Genre, Length: 104, dtype: int64

## CLEANING IMDb_Score

#### Read in IMDb_Score Table

In [11]:
score_df = pd.read_csv("../Data/Old_Dirty_Data/IMDb_Score.csv")

In [12]:
score_df.head()

,Unnamed: 0,Score_ID,IMDb_Score,Audience_Reviews,US_Users,Non_US_Users,Male_Reviews,Female_Reviews,Male_Score,Female_Score
0,0,7041,9.3,2666904,566032,1672983,1475251,294889,9.3,9.2
1,1,7042,9.2,1848055,417566,1161734,1072921,166441,9.2,8.9
2,2,7043,9.0,2639814,621027,1646769,1490958,267216,9.0,8.7
3,3,7044,9.0,1265636,282291,805848,759883,105690,9.0,8.7
4,4,7045,9.0,787589,187528,509017,453568,76860,9.0,8.9


Drop first column

In [13]:
score_df.pop("Unnamed: 0")

0        0
1        1
2        2
3        3
4        4
      ... 
245    245
246    246
247    247
248    248
249    249
Name: Unnamed: 0, Length: 250, dtype: int64

In [14]:
score_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Score_ID          250 non-null    int64  
 1   IMDb_Score        250 non-null    float64
 2   Audience_Reviews  250 non-null    int64  
 3   US_Users          250 non-null    int64  
 4   Non_US_Users      250 non-null    int64  
 5   Male_Reviews      250 non-null    int64  
 6   Female_Reviews    250 non-null    int64  
 7   Male_Score        250 non-null    float64
 8   Female_Score      250 non-null    float64
dtypes: float64(3), int64(6)
memory usage: 17.7 KB


Changing IMDb score to be out of 100 

In [15]:
score_df['IMDb_Score_Pct'] = score_df['IMDb_Score']*10

## Cleaning RT Table

#### Read in Rotten_Tomatoes Table

In [16]:
rt_df = pd.read_csv('../Data/Old_Dirty_Data/Rotten_Tomatoes.csv')

In [17]:
rt_df

,Unnamed: 0,Title_ID,Tomatometer,RT_Audience_Score,RT_Critic_Reviews,RT_Audience_Reviews,US_Box_Office,Distributor,Original_Language
0,0,17761,91.0,98.0,82,"250,000+",27.3 M,Columbia Pictures,English
1,1,17762,97.0,98.0,149,"250,000+",134.8 M,Paramount Pictures,English
2,2,17763,94.0,94.0,345,"250,000+",2.0 M,Warner Bros.,English
3,3,17764,96.0,97.0,123,"250,000+",0,Paramount Pictures,English
4,4,17765,100.0,97.0,58,"100,000+",0,"Criterion Collection, ...",English
...,...,...,...,...,...,...,...,...,...
245,245,1776246,73.0,94.0,15,"5,000+",0,New World Pictures,Russian
246,246,1776247,57.0,94.0,383,"50,000+ Verified",355.6 M,Walt Disney,English
247,247,1776248,76.0,89.0,232,"100,000+",169.7 M,Walt Disney,English
248,248,1776249,96.0,90.0,143,"100,000+",0,Warner Bros. Pictures,English


In [18]:
rt_df.pop("Unnamed: 0")

0        0
1        1
2        2
3        3
4        4
      ... 
245    245
246    246
247    247
248    248
249    249
Name: Unnamed: 0, Length: 250, dtype: int64

In [19]:
rt_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Title_ID             250 non-null    int64  
 1   Tomatometer          248 non-null    float64
 2   RT_Audience_Score    249 non-null    float64
 3   RT_Critic_Reviews    250 non-null    int64  
 4   RT_Audience_Reviews  250 non-null    object 
 5   US_Box_Office        250 non-null    object 
 6   Distributor          250 non-null    object 
 7   Original_Language    250 non-null    object 
dtypes: float64(2), int64(2), object(4)
memory usage: 15.8+ KB


#### clean audience reviews (ex. 250,000 + to 250K+)

In [20]:
rt_df['RT_Audience_Reviews'].value_counts()

250,000+            101
100,000+             37
50,000+              32
25,000+              30
10,000+              23
5,000+               11
1,000+                4
50,000+ Verified      3
25,000+ Verified      2
5,000+ Verified       1
Fewer than 50         1
2,500+                1
10,000+ Verified      1
0                     1
500+                  1
2,500+ Verified       1
Name: RT_Audience_Reviews, dtype: int64

In [21]:
rt_df['RT_Audience_Reviews'] = [x.replace(" Verified","") for x in rt_df['RT_Audience_Reviews']]

In [22]:
rt_df['RT_Audience_Reviews'] = [x.replace(",000+","k+").replace(",500+",".5k+") for x in rt_df['RT_Audience_Reviews']]

In [23]:
rt_df['RT_Audience_Reviews'].value_counts()

250k+            101
100k+             37
50k+              35
25k+              32
10k+              24
5k+               12
1k+                4
2.5k+              2
Fewer than 50      1
0                  1
500+               1
Name: RT_Audience_Reviews, dtype: int64

#### Averaging out Critics and Audience score to be more comparable to IMDb 

In [24]:
rt_df['RT_All_Pct'] = rt_df[['Tomatometer','RT_Audience_Score']].mean(axis=1)

In [25]:
rt_df

,Title_ID,Tomatometer,RT_Audience_Score,RT_Critic_Reviews,RT_Audience_Reviews,US_Box_Office,Distributor,Original_Language,RT_All_Pct
0,17761,91.0,98.0,82,250k+,27.3 M,Columbia Pictures,English,94.5
1,17762,97.0,98.0,149,250k+,134.8 M,Paramount Pictures,English,97.5
2,17763,94.0,94.0,345,250k+,2.0 M,Warner Bros.,English,94.0
3,17764,96.0,97.0,123,250k+,0,Paramount Pictures,English,96.5
4,17765,100.0,97.0,58,100k+,0,"Criterion Collection, ...",English,98.5
...,...,...,...,...,...,...,...,...,...
245,1776246,73.0,94.0,15,5k+,0,New World Pictures,Russian,83.5
246,1776247,57.0,94.0,383,50k+,355.6 M,Walt Disney,English,75.5
247,1776248,76.0,89.0,232,100k+,169.7 M,Walt Disney,English,82.5
248,1776249,96.0,90.0,143,100k+,0,Warner Bros. Pictures,English,93.0


#### Clean Distributor to remove spaces for movies with multiple distributors

In [26]:
list(rt_df['Distributor'])

['Columbia Pictures',
 'Paramount Pictures',
 'Warner Bros.',
 'Paramount Pictures',
 'Criterion Collection,                                                 MGM/UA Home Entertainment Inc.,                                                 United Artists,                                                 Metro-Goldwyn-Mayer',
 'Universal Pictures',
 'New Line Cinema',
 'Miramax Films',
 'Warner Bros.,                                                 New Line Cinema,                                                 Argentina Video Home,                                                 FS Film Oy',
 'Metro-Goldwyn-Mayer,                                                 United Artists',
 'Paramount Pictures',
 '20th Century Fox',
 'New Line Cinema',
 'Warner Bros. Pictures',
 '20th Century Fox',
 'Warner Bros. Pictures',
 'Warner Bros. Pictures',
 'Republic Pictures,                                                 United Artists,                                                 Pioneer Entertainmen

In [27]:
for x in range(len(rt_df['Distributor'])):
    try:
        rt_df['Distributor'][x] = re.sub(r",\s*",", ", rt_df['Distributor'][x])
    except:
        rt_df['Distributor'][x]

C:\Users\hanna\AppData\Local\Temp\ipykernel_16924\3607130934.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rt_df['Distributor'][x] = re.sub(r",\s*",", ", rt_df['Distributor'][x])


In [28]:
list(rt_df['Distributor'])

['Columbia Pictures',
 'Paramount Pictures',
 'Warner Bros.',
 'Paramount Pictures',
 'Criterion Collection, MGM/UA Home Entertainment Inc., United Artists, Metro-Goldwyn-Mayer',
 'Universal Pictures',
 'New Line Cinema',
 'Miramax Films',
 'Warner Bros., New Line Cinema, Argentina Video Home, FS Film Oy',
 'Metro-Goldwyn-Mayer, United Artists',
 'Paramount Pictures',
 '20th Century Fox',
 'New Line Cinema',
 'Warner Bros. Pictures',
 '20th Century Fox',
 'Warner Bros. Pictures',
 'Warner Bros. Pictures',
 'Republic Pictures, United Artists, Pioneer Entertainment, Warner Home Vídeo',
 'Criterion Collection, New Line Cinema',
 'Columbia Pictures',
 'RKO Radio Pictures',
 'Orion Pictures',
 'Miramax Films',
 'Paramount Pictures, DreamWorks SKG',
 'Miramax Films',
 'Paramount Pictures',
 'Warner Home Vídeo, Warner Bros., Warner Bros. Pictures',
 'Not Available',
 'TriStar Pictures, Pioneer Entertainment, Artisan Entertainment, Live Video',
 'Universal Pictures',
 'Walt Disney Pictures',
 

Need to fill nulls with 0 (which represented "not available") because I need the column as INT

In [29]:
rt_df['Tomatometer'] = rt_df['Tomatometer'].replace(np.nan, 0)

In [30]:
rt_df['RT_Audience_Score'] = rt_df['RT_Audience_Score'].replace(np.nan, 0)

In [31]:
rt_df['RT_Audience_Score'] = rt_df['RT_Audience_Score'].astype(int)

In [32]:
rt_df['Tomatometer'] = rt_df['Tomatometer'].astype(int)

In [33]:
score_df['IMDb_Score_Pct'] = score_df['IMDb_Score_Pct'].astype(int)

US BOX OFFICE

In [34]:
US_Box = []
Box_Label = []

for x in rt_df['US_Box_Office']:
        y = x.split(" ")
        US_Box.append(y[0])
        Box_Label.append(y[-1])

print("US Box Office Base Integers")
print(US_Box)
print(" ")
print("Box Office Amount Labels")
print(Box_Label)

US Box Office Base Integers
['27.3', '134.8', '2.0', '0', '0', '96.6', '377.8', '0', '315.5', '0', '330.3', '37.0', '342.6', '292.6', '291.9', '171.4', '42.6', '0', '100.1', '271.7', '3.3', '130.7', '7.6', '0', '57.6', '188.0', '136.8', '345', '552.8', '247.0', '10.0', '0', '32.5', '53.4', '19.0', '422.8', '187.7', '6.7', '132.4', '22.6', '53.1', '13.2', '0', '0', '13.2', '0', '163.2', '0', '1.6', '9.1', '81.1', '18.2', '78.8', '25.5', '248.2', '162.8', '0', '11.3', '0', '0', '0', '0', '678.8', '0', '77.6', '190.2', '130.1', '0', '448.1', '707.4', '0', '335.4', '120.5', '210.5', '31.3', '75.5', '0', '858.4', '0', '0', '138.3', '717.7', '4.7', '3.6', '6.5', '23.9', '415.0', '0', '309.3', '1.7', '1.3', '34.1', '2.5', '611.0', '70.8', '977.3', '29.6', '6.0', '0', '1.8', '33.2', '0', '0', '55.2', '0', '42.9', '0', '656.2', '0', '0', '6.9', '0', '66.2', '0', '331.7', '1.2', '64.6', '30.1', '81.0', '332.9', '195.1', '1.2', '159.2', '5.5', '0', '12.4', '206.9', '0', '0', '2.1', '10.2', '85.1'

In [35]:
#add new columns to rt
rt_df['US_Box'] = US_Box
rt_df['Box_Label'] = Box_Label

# turn old column into float/numeric
rt_df['US_Box'] = rt_df['US_Box'].astype(float)

In [36]:
# initiate empty column for math
rt_df['Box_Int'] = ''

# for loop using iterrows and if/ else to say if m 

for ind, row in rt_df.iterrows():
    if row['Box_Label'] == 'M':
        rt_df.loc[ind, 'Box_Int'] = 100000
    elif row['Box_Label'] == 'K':
        rt_df.loc[ind, 'Box_Int'] = 1000
    else:
        rt_df.loc[ind, 'Box_Int'] = 1

In [37]:
# then multiply by a million, if k then multiply by 1000 and add to new column
rt_df['US_Box_Int'] = rt_df['US_Box']*rt_df['Box_Int']

In [38]:
rt_df.pop('US_Box')
rt_df.pop('Box_Int')
rt_df.pop('Box_Label')

0      M
1      M
2      M
3      0
4      0
      ..
245    0
246    M
247    M
248    0
249    0
Name: Box_Label, Length: 250, dtype: object

#### NEW GENRE DATAFRAME

In [39]:
genre1 = movie_df[['Title_ID','Genre']]

In [40]:
Genre = (genre1.set_index(['Title_ID']).apply(lambda x: x.str.split(', ').explode()).reset_index())                                                   

#### NEW DISTRIBUTOR DATAFRAME

In [41]:
Distributor = (rt_df[['Title_ID','Distributor']].set_index(['Title_ID']).apply(lambda x: x.str.split(', ').explode()).reset_index())

In [42]:
movie_df.pop('Genre')
rt_df.pop('Distributor')

0                                      Columbia Pictures
1                                     Paramount Pictures
2                                           Warner Bros.
3                                     Paramount Pictures
4      Criterion Collection, MGM/UA Home Entertainmen...
                             ...                        
245                                   New World Pictures
246                                          Walt Disney
247                                          Walt Disney
248                                Warner Bros. Pictures
249                                    Columbia Pictures
Name: Distributor, Length: 250, dtype: object

### with cleaned up tables, I need to save them

#### Saving as CSV

In [43]:
movie_df.to_csv('../Data/IMDb_Movies.csv', index = False)

In [44]:
score_df.to_csv('../Data/IMDb_Score.csv', index = False)

In [45]:
rt_df.to_csv('../Data/Rotten_Tomatoes.csv', index = False)

In [46]:
Genre.to_csv('../Data/Genre.csv', index = False)

In [47]:
Distributor.to_csv('../Data/Distributor.csv', index = False)